In [ ]:
### Импорт библиотек и настройки
import glob
from collections import namedtuple
import pandas as pd
import numpy as np
import scipy.stats as ss
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from IPython.display import display

pd.set_option("display.precision", 3)
pd.set_option("display.max_columns", 20)
sns.set(style="whitegrid", palette="pastel")
%matplotlib inline

# ### Загрузка и предобработка данных
def load_and_preprocess_data(experiment_name: str, data_pattern: str) -> pd.DataFrame:
    """Загрузка и объединение данных экспериментов"""
    files = glob.glob(data_pattern)
    if not files:
        raise ValueError(f"No files found matching pattern: {data_pattern}")
    
    df = pd.concat([pd.read_json(f, lines=True) for f in files])
    df["treatment"] = df["experiments"].map(lambda x: x.get(experiment_name, "unknown"))
    
    # Фильтрация и проверка данных
    initial_shape = df.shape[0]
    df = df[df["treatment"].isin(["C", "T1"])]  # Оставляем только известные группы
    print(f"Filtered {initial_shape - df.shape[0]} rows with unknown treatments")
    
    return df

experiment = "rec"
data = load_and_preprocess_data(
    experiment,
    "../../rec_sys_data/hw_2/rec/*/data.json"
)

# ### Визуализация распределения данных
def plot_treatment_distribution(df: pd.DataFrame):
    """Визуализация распределения пользователей по группам"""
    plt.figure(figsize=(10, 6))
    treatment_counts = df.groupby("treatment")["user"].nunique().reset_index()
    sns.barplot(x="treatment", y="user", data=treatment_counts)
    plt.title("Распределение пользователей по группам")
    plt.ylabel("Количество пользователей")
    plt.show()

plot_treatment_distribution(data)

# ### Анализ сессий пользователей
Session = namedtuple("Session", ["timestamp", "tracks", "time", "latency"])

def sessionize(user_data: pd.DataFrame) -> list:
    """Создание сессий из сырых событий"""
    sessions = []
    current_session = None
    
    for _, row in user_data.sort_values("timestamp").iterrows():
        if current_session is None:
            current_session = Session(
                timestamp=row["timestamp"],
                tracks=0,
                time=0.0,
                latency=0.0
            )
        
        current_session = current_session._replace(
            tracks=current_session.tracks + 1,
            time=current_session.time + row["time"],
            latency=current_session.latency + row["latency"] * 1000
        )
        
        if row["message"] == "last":
            sessions.append(current_session._asdict())
            current_session = None
    
    return sessions

# Создание датафрейма сессий
sessions = (
    data
    .groupby(["user", "treatment"], group_keys=False)
    .apply(sessionize)
    .explode()
    .apply(pd.Series)
    .reset_index()
)

### Расширенная визуализация метрик сессий
def plot_session_metrics(df: pd.DataFrame):
    """Интерактивные графики метрик сессий"""
    fig = px.box(
        df,
        x="treatment",
        y="tracks",
        title="Распределение количества треков в сессиях",
        color="treatment"
    )
    fig.show()

    fig = px.scatter(
        df,
        x="time",
        y="latency",
        color="treatment",
        trendline="lowess",
        title="Зависимость времени сессии от задержки"
    )
    fig.show()

plot_session_metrics(sessions)

### Расширенный анализ на уровне пользователей
def calculate_user_metrics(session_df: pd.DataFrame) -> pd.DataFrame:
    """Расчет пользовательских метрик"""
    user_metrics = (
        session_df
        .groupby(["user", "treatment"])
        .agg(
            total_sessions=("timestamp", "count"),
            total_tracks=("tracks", "sum"),
            total_time=("time", "sum"),
            avg_latency=("latency", "mean")
        )
        .reset_index()
    )
    
    user_metrics["tracks_per_session"] = user_metrics["total_tracks"] / user_metrics["total_sessions"]
    user_metrics["time_per_session"] = user_metrics["total_time"] / user_metrics["total_sessions"]
    
    return user_metrics

user_metrics = calculate_user_metrics(sessions)

### Статистический анализ метрик
def analyze_metric(control_data, treatment_data, metric_name: str, alpha=0.05):
    """Расширенный статистический анализ для одной метрики"""
    # Проверка нормальности распределения
    _, p_normal = ss.shapiro(control_data)
    is_normal = p_normal > alpha
    
    # Выбор теста в зависимости от распределения
    if is_normal:
        test_stat, p_value = ss.ttest_ind(control_data, treatment_data)
        test_name = "t-тест"
    else:
        test_stat, p_value = ss.mannwhitneyu(control_data, treatment_data)
        test_name = "U-тест Манна-Уитни"
    
    # Расчет размера эффекта
    effect_size = (treatment_data.mean() - control_data.mean()) / control_data.std()
    
    return {
        "metric": metric_name,
        "test": test_name,
        "p_value": p_value,
        "effect_size": effect_size,
        "is_significant": p_value < alpha,
        "control_mean": control_data.mean(),
        "treatment_mean": treatment_data.mean()
    }

# Анализ основных метрик
metrics_to_analyze = [
    "tracks_per_session",
    "time_per_session",
    "avg_latency",
    "total_sessions"
]

results = []
for metric in metrics_to_analyze:
    control = user_metrics[user_metrics["treatment"] == "C"][metric]
    treatment = user_metrics[user_metrics["treatment"] == "T1"][metric]
    
    results.append(analyze_metric(control, treatment, metric))

### Визуализация результатов анализа
def plot_statistical_results(results: list):
    """Визуализация результатов статистического анализа"""
    df = pd.DataFrame(results)
    
    fig, ax = plt.subplots(figsize=(12, 6))
    sns.barplot(
        x="metric",
        y="effect_size",
        hue="is_significant",
        data=df,
        ax=ax
    )
    ax.set_title("Размер эффекта по метрикам")
    ax.set_ylabel("Размер эффекта (SD units)")
    ax.axhline(0, color="black", linestyle="--")
    plt.xticks(rotation=45)
    plt.show()

plot_statistical_results(results)

### Интерактивная таблица результатов
def format_results_table(results: list) -> pd.DataFrame.style:
    """Стилизация таблицы результатов"""
    df = pd.DataFrame(results)
    format_dict = {
        "p_value": "{:.4f}",
        "effect_size": "{:.2f}",
        "control_mean": "{:.2f}",
        "treatment_mean": "{:.2f}"
    }
    
    return (
        df.style
        .format(format_dict)
        .bar(subset=["effect_size"], color="#d65f5f")
        .applymap(lambda x: "color: green" if x < 0.05 else "color: black", 
                subset=["p_value"])
    )

display(format_results_table(results))


,treatment,metric_name,control_value,treatment_value,significant
0,t1,session,15.56,17.20,True
1,t1,mean_track_per_session,8.06,8.51,True
2,t1,time,44.82,50.01,True
